In [3]:
import glob
import pandas as pd
import matplotlib.pyplot as plt

import pyproj
from tqdm import tqdm
import folium
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import numpy as np
import random
import haversine as hs
from multiprocessing import Pool
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn import neighbors
from sklearn.linear_model import ElasticNet
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 修改中文字體
plt.rcParams['axes.unicode_minus'] = False # 顯示負號
pd.set_option('display.max_columns', None)

In [4]:
df_train = pd.read_csv('../官方資料集/training_data.csv')
df_valid = pd.read_csv('../官方資料集/public_dataset.csv')
df_test = pd.read_csv('../官方資料集/private_dataset.csv')

In [5]:
df_external_gov_data = pd.read_csv('../外部資料集/實價登錄/external_gov_data.csv')

In [6]:
by = ['縣市', '鄉鎮市區', '路名', '主要用途', '建物型態']
df_train['key'] = df_train[by].apply(lambda x: '_'.join([str(v) for v in x]), axis=1)
df_train['key']

0        台北市_大安區_敦化南路二段_住家用_住宅大樓(11層含以上有電梯)
1           台北市_萬華區_水源路_住家用_住宅大樓(11層含以上有電梯)
2          高雄市_鳳山區_北忠街_集合住宅_住宅大樓(11層含以上有電梯)
3          新北市_新莊區_福前街_集合住宅_住宅大樓(11層含以上有電梯)
4         新北市_板橋區_文化路一段_住家用_住宅大樓(11層含以上有電梯)
                        ...                
11746        桃園市_八德區_介壽路二段_住家用_公寓(5樓含以下無電梯)
11747         新竹市_新竹市_東南街_住家用_華廈(10層含以下有電梯)
11748       新北市_汐止區_建成路_住家用_住宅大樓(11層含以上有電梯)
11749       新北市_土城區_學士路_住家用_住宅大樓(11層含以上有電梯)
11750     新北市_三重區_大同南路_集合住宅_住宅大樓(11層含以上有電梯)
Name: key, Length: 11751, dtype: object

In [7]:
# external datas 
externalkey2subdf = {}
key_col = 'key'
_new_col_name = ''

def get_external_same_building_feature(row):
    global _new_col_name
    if row[key_col] not in externalkey2subdf:
        return {}
    subdf = externalkey2subdf[row[key_col]]
    if len(subdf) == 0:
        return {}
    else:
        return {
            'ID': row['ID'],
            f'{_new_col_name}_mean': subdf['單價'].mean(),
            f'{_new_col_name}_std': subdf['單價'].std(),
            f'{_new_col_name}_max': subdf['單價'].max(),
            f'{_new_col_name}_min': subdf['單價'].min(),
            f'{_new_col_name}_max_min_ratio': (subdf['單價'].max()-subdf['單價'].min()) / subdf['單價'].mean(),
        }
        
def mapping_external_gov_data_price(
    df_train, 
    df_valid, 
    df_external_gov_data, 
    by = ['縣市', '鄉鎮市區', '路名', '主要用途', '建物型態'], 
    new_col_name = 'externalkey_sameroad_price'):
    global _new_col_name
    _new_col_name = new_col_name
    
    df_train[key_col] = df_train[by].apply(lambda x: '_'.join([str(v) for v in x]), axis=1)
    df_valid[key_col] = df_valid[by].apply(lambda x: '_'.join([str(v) for v in x]), axis=1)
    df_external_gov_data[key_col] = df_external_gov_data[by].apply(lambda x: '_'.join([str(v) for v in x]), axis=1)
    
    le = LabelEncoder()
    le.fit(df_train[key_col].values.tolist() + df_valid[key_col].values.tolist() + df_external_gov_data[key_col].values.tolist())
    df_train[key_col] = le.transform(df_train[key_col].values.tolist())
    df_valid[key_col] = le.transform(df_valid[key_col].values.tolist())
    df_external_gov_data[key_col] = le.transform(df_external_gov_data[key_col].values.tolist())


    
    global externalkey2subdf
    externalkey2subdf = {}
    for key, subdf in df_external_gov_data.groupby(key_col):
        externalkey2subdf[key] = subdf
    
    with Pool(22) as pool:
        features = list(tqdm(pool.imap(get_external_same_building_feature, df_train.to_dict('records')), total=len(df_train)))
    df_train_features = pd.DataFrame(features)
    df_train = df_train.merge(df_train_features, how='left', on='ID')
    
    with Pool(22) as pool:
        features = list(tqdm(pool.imap(get_external_same_building_feature, df_valid.to_dict('records')), total=len(df_valid)))
    df_valid_features = pd.DataFrame(features)
    df_valid = df_valid.merge(df_valid_features, how='left', on='ID')
    return df_train, df_valid

In [8]:
df_train, df_valid = mapping_external_gov_data_price(
    df_train, 
    df_valid, 
    df_external_gov_data, 
    by = ['縣市', '鄉鎮市區', '路名'], 
    new_col_name = 'external_key1_price'
)
na_cnt = sum(df_train['external_key1_price_mean'].isna())
mapping_rate = 1 - na_cnt / len(df_train)
print(f'mapping_rate = {round(mapping_rate*100, 3)}%')
df_train[['單價', 'external_key1_price_mean']].corr()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5876/5876 [00:00<00:00, 6573.05it/s]


mapping_rate = 97.575%


,單價,external_key1_price_mean
單價,1.000000,0.895518
external_key1_price_mean,0.895518,1.000000


In [9]:
df_train, df_valid = mapping_external_gov_data_price(
    df_train, 
    df_valid, 
    df_external_gov_data, 
    by = ['縣市', '鄉鎮市區', '路名', '主要用途', '建物型態'], 
    new_col_name = 'external_key2_price'
)
na_cnt = sum(df_train['external_key2_price_mean'].isna())
mapping_rate = 1 - na_cnt / len(df_train)
print(f'mapping_rate = {round(mapping_rate*100, 3)}%')
df_train[['單價', 'external_key2_price_mean']].corr()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5876/5876 [00:00<00:00, 6401.30it/s]

mapping_rate = 91.099%


,單價,external_key2_price_mean
單價,1.000000,0.926038
external_key2_price_mean,0.926038,1.000000


In [10]:
df_train, df_valid = mapping_external_gov_data_price(
    df_train, 
    df_valid, 
    df_external_gov_data, 
    by = ['縣市', '鄉鎮市區', '路名', '主要用途', '建物型態', '總樓層數'], 
    new_col_name = 'external_key3_price'
)
na_cnt = sum(df_train['external_key3_price_mean'].isna())
mapping_rate = 1 - na_cnt / len(df_train)
print(f'mapping_rate = {round(mapping_rate*100, 3)}%')
df_train[['單價', 'external_key3_price_mean']].corr()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5876/5876 [00:00<00:00, 6040.60it/s]


mapping_rate = 87.584%


,單價,external_key3_price_mean
單價,1.000000,0.948541
external_key3_price_mean,0.948541,1.000000


In [9]:
df_train, df_valid = mapping_external_gov_data_price(
    df_train, 
    df_valid, 
    df_external_gov_data, 
    by = ['縣市', '鄉鎮市區', '路名', '主要用途', '建物型態', '總樓層數', '移轉層次'], 
    new_col_name = 'external_key4_price'
)
na_cnt = sum(df_train['external_key4_price_mean'].isna())
mapping_rate = 1 - na_cnt / len(df_train)
print(f'mapping_rate = {round(mapping_rate*100, 3)}%')
df_train[['單價', 'external_key4_price_mean']].corr()

100%|█████████████████████████████████████████████████████████████████████████████| 5876/5876 [00:00<00:00, 5996.57it/s]


mapping_rate = 66.505%


,單價,external_key4_price_mean
單價,1.000000,0.935868
external_key4_price_mean,0.935868,1.000000


In [11]:
df_train['附屬建物面積'].value_counts().reset_index().head()

,附屬建物面積,count
0,-0.438452,6174
1,-0.118180,24
2,-0.171559,21
3,-0.062666,20
4,0.041956,20


In [12]:
df_valid['附屬建物面積'].value_counts().reset_index().head()

,附屬建物面積,count
0,-0.438452,3129
1,-0.171559,13
2,0.020604,11
3,-0.118180,10
4,-0.101099,10


In [13]:
df_test['附屬建物面積'].value_counts().reset_index().head()

,附屬建物面積,count
0,-0.438452,3087
1,-0.242019,15
2,-0.171559,14
3,-0.062666,12
4,-0.079748,10


In [15]:
df_external_gov_data['附屬建物面積'].value_counts().reset_index().head()

,附屬建物面積,count
0,-0.651622,117855
1,0.189057,535
2,0.179147,434
3,0.258440,404
4,0.053309,404


In [16]:
df_extra = pd.read_csv('../官方資料集/external_data/國小基本資料.csv')
df_extra.head(1)

,縣市代碼,縣市名稱,學校代碼,學校名稱,1年級班級數,2年級班級數,3年級班級數,4年級班級數,5年級班級數,6年級班級數,1年級男學生數,1年級女學生數,2年級男學生數,2年級女學生數,3年級男學生數,3年級女學生數,4年級男學生數,4年級女學生數,5年級男學生數,5年級女學生數,6年級男學生數,6年級女學生數,上學年男畢業生,上學年女畢業生,男專任教師,女專任教師,男職員,女職員,lat,lng
0,1,新北市,11301,私立淡江高中附設國小部,3,3,2,2,2,3,53,52,41,48,32,38,38,28,37,27,42,40,45,46,10,14,0,2,25.174899,121.435956


In [17]:
df_extra = pd.read_csv('../官方資料集/external_data/國中基本資料.csv')
df_extra.head(1)

,縣市代碼,縣市名稱,學校代碼,學校名稱,班級數7年級,班級數8年級,班級數9年級,學生數7年級男,學生數7年級女,學生數8年級男,學生數8年級女,學生數9年級男,學生數9年級女,上學年男畢業生,上學年女畢業生,男專任教師,女專任教師,男職員,女職員,lat,lng
0,1,新北市,10301,國立華僑中學,1,1,1,2,2,5,6,9,11,9,16,0,0,0,0,25.0075,121.447778


In [18]:
df_extra = pd.read_csv('../官方資料集/external_data/大學基本資料.csv')
df_extra.head(1)

,學校代碼,學校名稱,日間∕進修別,等級別,總計,男生計,女生計,一年級男生,一年級女生,二年級男生,二年級女生,三年級男生,三年級女生,四年級男生,四年級女生,五年級男生,五年級女生,六年級男生,六年級女生,七年級男生,七年級女生,延修生男生,延修生女生,縣市名稱,體系別,lat,lng
0,23,國立雲林科技大學,D 日,B 四技,6287,3887,2400,988,558,993,573,897,596,848,608,0,0,0,0,0,0,161,65,09 雲林縣,2 技職,23.69606,120.534043


In [19]:
pd.set_option('display.max_columns', None)
df_dataset = pd.read_csv('../官方資料集/public_dataset.csv')
df_dataset.sample()

,ID,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,建物型態,屋齡,建物面積,車位面積,車位個數,橫坐標,縱坐標,備註,主建物面積,陽台面積,附屬建物面積
1007,PU-1008,台北市,萬華區,東園街,-0.194109,NaN,4,5,住家用,鋼筋混凝土造,公寓(5樓含以下無電梯),41.5,-0.849811,-0.819326,0.0,300034,2768753,NaN,-0.543859,0.344095,-0.438452


In [20]:
df_train = pd.read_csv('../官方資料集/training_data.csv')

In [21]:
df_train['屋齡'].describe()

count    11751.000000
mean        21.319058
std         14.425560
min          0.000000
25%          8.083333
50%         21.750000
75%         31.250000
max         60.083333
Name: 屋齡, dtype: float64

In [22]:
df_train.sample()

,ID,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,建物型態,屋齡,建物面積,車位面積,車位個數,橫坐標,縱坐標,備註,主建物面積,陽台面積,附屬建物面積,單價
6358,TR-6359,台南市,安南區,郡安路四段,0.139814,NaN,7,11,住家用,鋼筋混凝土造,住宅大樓(11層含以上有電梯),27.916667,-0.217819,0.555549,1.0,169117,2547626,NaN,-0.171574,0.449888,-0.438452,1.020618


In [23]:
len(df_train.groupby(['縣市', '鄉鎮市區']))

129

In [24]:
df_train['主要用途'].unique()

array(['住家用', '集合住宅', '其他', '店鋪', '商業用', '國民住宅', '住工用', '一般事務所', '住商用',
       '廠房', '工業用', '辦公室'], dtype=object)

In [25]:
df_train['主要用途'].value_counts()

主要用途
住家用      8230
集合住宅     2660
其他        471
商業用       263
一般事務所      59
國民住宅       29
住商用        11
工業用        11
辦公室        11
住工用         3
店鋪          2
廠房          1
Name: count, dtype: int64

In [26]:
df_train.query('主要用途 == "住家用"')['單價'].describe()

count    8230.000000
mean        1.997874
std         1.029401
min         0.285145
25%         1.278268
50%         1.770145
75%         2.402558
max        13.622033
Name: 單價, dtype: float64

In [27]:
df_train.query('主要用途 == "集合住宅"')['單價'].describe()

count    2660.000000
mean        1.868529
std         0.734776
min         0.177415
25%         1.371959
50%         1.699877
75%         2.262021
max         9.921246
Name: 單價, dtype: float64

In [28]:
df_dataset['路名'].apply(lambda x: x[-1]).value_counts()

路名
路    2741
街    1636
段    1449
堡       6
道       5
巷       5
埔       4
寮       3
口       3
勢       2
下       2
前       2
莊       1
塘       1
州       1
苗       1
羅       1
化       1
分       1
星       1
碓       1
興       1
林       1
富       1
山       1
屯       1
心       1
城       1
村       1
愛       1
Name: count, dtype: int64

In [29]:
df_dataset['縣市'].unique()

array(['新北市', '台北市', '桃園市', '高雄市', '台中市', '新竹縣', '新竹市', '基隆市', '屏東縣',
       '台南市', '宜蘭縣', '苗栗縣', '嘉義市', '金門縣', '花蓮縣', '彰化縣', '雲林縣'],
      dtype=object)

In [30]:
df_dataset['縣市'].value_counts()

縣市
新北市    2197
台北市    1219
高雄市     669
桃園市     549
台中市     485
台南市     214
新竹縣     167
新竹市     143
基隆市     100
宜蘭縣      30
苗栗縣      28
金門縣      20
嘉義市      20
屏東縣      19
彰化縣       7
花蓮縣       6
雲林縣       3
Name: count, dtype: int64

In [31]:
df_dataset['縣市'].nunique()

17

In [35]:
df_train = pd.read_csv('../官方資料集/training_data.csv')
for col in df_train.columns:
    print(df_train[col].describe())
    print()

count     11751
unique    11751
top        TR-1
freq          1
Name: ID, dtype: object

count     11751
unique       18
top         新北市
freq       4302
Name: 縣市, dtype: object

count     11751
unique      123
top         板橋區
freq        560
Name: 鄉鎮市區, dtype: object

count     11751
unique     3059
top         中正路
freq        165
Name: 路名, dtype: object

count    11751.000000
mean         0.018416
std          1.072279
min         -1.619755
25%         -0.646264
50%         -0.149839
75%          0.410167
max         19.475175
Name: 土地面積, dtype: float64

count     595
unique      5
top         住
freq      308
Name: 使用分區, dtype: object

count    11751.000000
mean         7.433665
std          5.089029
min          2.000000
25%          4.000000
50%          6.000000
75%         10.000000
max         46.000000
Name: 移轉層次, dtype: float64

count    11751.000000
mean        12.602672
std          6.840007
min          2.000000
25%          7.000000
50%         12.000000
75%         15.0000

In [36]:
df_train['車位面積'].value_counts()

車位面積
-0.819326    5624
 0.725477      24
 0.503026      22
 0.574087      21
 0.657506      20
             ... 
-0.545896       1
-0.408408       1
 3.365546       1
 1.195098       1
 3.220335       1
Name: count, Length: 1760, dtype: int64

In [37]:
df_train.columns

Index(['ID', '縣市', '鄉鎮市區', '路名', '土地面積', '使用分區', '移轉層次', '總樓層數', '主要用途',
       '主要建材', '建物型態', '屋齡', '建物面積', '車位面積', '車位個數', '橫坐標', '縱坐標', '備註',
       '主建物面積', '陽台面積', '附屬建物面積', '單價'],
      dtype='object')

In [38]:
df_train['縣市'].value_counts()

縣市
新北市    4302
台北市    2382
高雄市    1328
桃園市    1178
台中市     996
台南市     478
新竹縣     347
新竹市     246
基隆市     233
宜蘭縣      76
屏東縣      46
苗栗縣      43
金門縣      30
嘉義市      27
彰化縣      23
花蓮縣      12
嘉義縣       2
雲林縣       2
Name: count, dtype: int64

In [39]:
df_train['縣市'].unique()

array(['台北市', '高雄市', '新北市', '桃園市', '台中市', '台南市', '苗栗縣', '新竹縣', '基隆市',
       '屏東縣', '新竹市', '宜蘭縣', '花蓮縣', '嘉義市', '金門縣', '嘉義縣', '彰化縣', '雲林縣'],
      dtype=object)

In [40]:
df_train['主要建材'].value_counts()

主要建材
鋼筋混凝土造       10923
鋼骨造            419
加強磚造           251
其他             145
鋼筋混凝土加強磚造       12
磚造               1
Name: count, dtype: int64

In [41]:
df_train['建物型態'].value_counts()

建物型態
住宅大樓(11層含以上有電梯)    7148
公寓(5樓含以下無電梯)       2437
華廈(10層含以下有電梯)      2158
透天厝                   8
Name: count, dtype: int64

In [42]:
df_train['備註'].value_counts().head(10)

備註
主要建材：鋼骨鋼筋混凝土造            3
花台0.79㎡                  2
花台1.54㎡                  2
門牌登載為8樓實際為7樓             2
花台2.65㎡                  1
露台1.23㎡、暸望室6.31㎡         1
花台0.98㎡                  1
陽台10.08、花台0.61、露樑0.13    1
花台1.4平方公尺                1
露臺4.4                    1
Name: count, dtype: int64

In [43]:
df_train['屋齡'].value_counts().head(10)

屋齡
0.166667     106
0.333333      78
0.250000      74
0.416667      65
0.583333      56
27.333333     54
0.083333      53
0.500000      53
27.833333     49
26.916667     48
Name: count, dtype: int64

In [49]:
import pyproj
twd97 = pyproj.Proj(init='epsg:3826')  # TWD97
wgs84 = pyproj.Proj(init='epsg:4326')  # WGS84

df_train = pd.read_csv('../官方資料集/training_data.csv')

In [50]:
def get_coordinate(row):
    lon, lat = pyproj.transform(twd97, wgs84, row['橫坐標'], row['縱坐標'])
    return {
        'ID': row['ID'],
        'Lon': lon,
        'Lat': lat
    }

with Pool(22) as pool:
    features = list(tqdm(pool.imap(get_coordinate, df_train.to_dict('records')), total=len(df_train)))
df_train_features = pd.DataFrame(features).fillna(-999999)
df_train = df_train.merge(df_train_features, how='left', on='ID')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11751/11751 [00:16<00:00, 697.81it/s]


In [52]:
import folium
import json
m = folium.Map(location=(df_train['Lat'].values[0], df_train['Lon'].values[0]), zoom_start=17)
for i, row in tqdm(df_train.iterrows()):
    folium.Marker(
        location=[row['Lat'], row['Lon']],
        popup=row.to_dict(),
        icon=folium.Icon(icon="info"),
    ).add_to(m)
    if i > 100:
        break
m

101it [00:00, 6010.40it/s]
